## RAG Application - Hindi

## 1. Setup

Importing Libraries

In [1]:
! pip install langchain_openai langchain whisper langchain_community scikit-learn langchain_pinecone langchain[docarray] docarray pydantic==1.10.8 pytube  python-dotenv tiktoken ruff --quiet pandas

Loading the environment variables we need to use.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Let's define the LLM model that we'll use as part of the workflow.

In [3]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

## 2. Loading the document

In [4]:
with open("kb_bn.txt", encoding="utf-8") as file:
    transcription = file.read()

transcription[:100]

'হস্তিনাপুরের প্রাচীন রাজ্যে, একটি মহান মহাকাব্য উদ্ভাসিত হয়েছিল, যা বিশ্বের ভাগ্যকে রূপ দেবে এবং নি'

## 3. Chunking the document

In [5]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("kb_bn.txt", encoding="utf-8")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'kb_bn.txt'}, page_content='হস্তিনাপুরের প্রাচীন রাজ্যে, একটি মহান মহাকাব্য উদ্ভাসিত হয়েছিল, যা বিশ্বের ভাগ্যকে রূপ দেবে এবং নিরবধি পাঠ দেবে। এই মহাকাব্যটি ছিল মহাভারত, দ্বন্দ্ব, কর্তব্য, ন্যায়পরায়ণতা এবং ভাগ্যের গল্প।\nরাজা শান্তনু শাসিত কুরু রাজবংশের সাথে গল্পটি শুরু হয়েছিল। শান্তনু নদী দেবী গঙ্গার সাথে বিবাহ বন্ধনে আবদ্ধ হন এবং তাদের দেবব্রত নামে একটি পুত্রের জন্ম হয়, যা পরে ভীষ্ম নামে পরিচিত হয়। ভীষ্ম তার পিতার সুখ নিশ্চিত করার জন্য ব্রহ্মচর্যের ব্রত নিয়েছিলেন, শান্তনুকে একজন জেলে সত্যবতীকে বিয়ে করার অনুমতি দিয়েছিলেন। সত্যবতী চিত্রাঙ্গদা ও বিচিত্রবীর্য নামে দুই পুত্রের জন্ম দেন। চিত্রাঙ্গদা অল্প বয়সেই মারা যান এবং বিচিত্রবীর্য রাজা হন। তিনি দুই রাজকন্যা অম্বিকা ও অম্বালিকাকে বিয়ে করেন, কিন্তু নিঃসন্তান মারা যান। বংশ অব্যাহত রাখার জন্য, ব্যাস, পূর্ববর্তী ইউনিয়নের সত্যবতীর পুত্র, নিয়োগের মাধ্যমে রাণীদের সাথে পুত্রের জন্ম দেন। এইভাবে, জন্মান্ধ ধৃতরাষ্ট্র এবং ফ্যাকাশে বর্ণের পাণ্ডুর জন্ম হয়।\nধৃতরাষ্ট্র গান্ধারীকে বিয়ে করেছিলেন, যিনি তার স্ব

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

## 4. Creating embeddings

In [22]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## 5. Setting up a Vector Store

In [23]:
from langchain_pinecone import PineconeVectorStore

index_name = "tmf1"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

--------------------------------------------------    INGESTION COMPLETES HERE     -----------------------------------------------------------

## 6. Chaining

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define the output parser
parser = StrOutputParser()

# Define the prompt template
template = """
প্রশ্নটি নিচের প্রসঙ্গের উপর ভিত্তি করে উত্তর দিন। যদি প্রশ্নের উত্তর না দিতে পারেন, "আমি জানি না" বলে উত্তর দিন।

    প্রসঙ্গ: {context}

    প্রশ্ন: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retrieved_contexts = []

# Function to log context
def log_context(inputs):
    context_content = [input.page_content for input in inputs['context']]
    retrieved_contexts.append(context_content)
    return inputs

# Chain the prompt, model, and parser
chain = {
    "context": pinecone.as_retriever(search_kwargs={"k": 2}), 
    "question": RunnablePassthrough()
} | RunnablePassthrough(log_context) | prompt | model | parser

## 7. Quering

In [26]:
import pandas as pd
df_1 = pd.read_csv('input_dataset.csv')
df = df_1[0:20:]

# Initialize lists for queries and responses
queries = df['question']
responses = []

In [27]:
df

,question,ground_truth,contexts,answer
0,মহাভারতের শুরুতে হস্তিনাপুরের শাসক কে ছিলেন?,রাজা শান্তনু।,NaN,NaN
1,ভীষ্ম কী ব্রত নিয়েছিলেন এবং কেন?,ভীষ্ম তার পিতার সুখ নিশ্চিত করতে এবং তার পিতা শান্তনুকে সত্যবতীকে বিয়ে করার অনুমতি দেওয়ার জন্য ব্রহ্মচর্যের ব্রত নিয়েছিলেন।,NaN,NaN
2,রাজা শান্তনু ও সত্যবতীর পুত্র কারা ছিলেন?,চিত্রাঙ্গদা ও বিচিত্রবীর্য।,NaN,NaN
3,ধৃতরাষ্ট্র ও পাণ্ডুর অস্তিত্ব কীভাবে হয়েছিল?,"ধৃতরাষ্ট্র এবং পাণ্ডুর জন্ম নিয়োগের মাধ্যমে, একটি অনুশীলন যেখানে ব্যাস বিচিত্রবীর্যের মৃত্যুর কারণে রাণী অম্বিকা এবং অম্বালিকার সাথে সন্তানের জন্ম দিয়েছিলেন।",NaN,NaN
4,মহাভারতে কুন্তীর ঐশ্বরিক আশীর্বাদের তাৎপর্য কী ছিল?,"কুন্তী দেবতাদের থেকে পুত্র সন্তান জন্ম দেওয়ার জন্য ঐশ্বরিক বর ব্যবহার করেছিলেন, এইভাবে পাণ্ডবদের জন্ম নিশ্চিত করেছিলেন: যুধিষ্ঠির, ভীম এবং অর্জুন।",NaN,NaN
5,পাঁচ পান্ডব কারা ছিলেন এবং তাদের মা কে ছিলেন?,"পাঁচ পাণ্ডব ছিলেন যুধিষ্ঠির (ধর্মের পুত্র), ভীম (বায়ুর পুত্র), অর্জুন (ইন্দ্রের পুত্র), নকুল এবং সহদেব (অশ্বিনী যমজের পুত্র)। কুন্তী ছিলেন যুধিষ্ঠির, ভীম ও অর্জুনের মা, আর মাদ্রী ছিলেন নকুল ও সহদেবের মা।",NaN,NaN
6,দ্রোণ কীভাবে কৌরব ও পাণ্ডব রাজকুমারদের শিক্ষক হলেন?,উন্নত সামরিক দক্ষতা প্রদানের জন্য দ্রোণকে তাদের শিক্ষক হিসাবে নিযুক্ত করা হয়েছিল।,NaN,NaN
7,কি কারণে অর্জুন ও কর্ণের মধ্যে শত্রুতা হয়েছিল?,"কর্ণ, প্রাথমিকভাবে একজন অজানা যোদ্ধা, দ্রৌপদীর স্বয়ম্বরে অর্জুনকে চ্যালেঞ্জ করেছিলেন। কর্ণের অসন্তোষ তার রাজকীয় রক্ত ​​সত্ত্বেও তার নিম্ন লালন-পালন থেকে উদ্ভূত হয়েছিল এবং কর্ণের প্রতি দুর্যোধনের সমর্থন প্রতিদ্বন্দ্বিতাকে আরও গভীর করে তোলে।",NaN,NaN
8,পাণ্ডবদের জীবনে দ্রৌপদীর ভূমিকা কী ছিল?,দ্রৌপদী তার স্বয়ম্বর সম্পর্কে ভুল বোঝাবুঝির কারণে পাঁচ পাণ্ডবেরই সাধারণ স্ত্রী হয়েছিলেন। কৌরব দরবারে তার অপমান ছিল একটি গুরুত্বপূর্ণ ঘটনা যা মহান যুদ্ধের দিকে নিয়ে যায়।,NaN,NaN
9,যুধিষ্ঠির ও দুর্যোধনের মধ্যে পাশা খেলার সময় কী ঘটেছিল?,"যুধিষ্ঠির তার রাজ্য, ভাই এবং দ্রৌপদী সহ দুর্যোধনের কাছে সবকিছু হারিয়েছিলেন, যা পাণ্ডবদের নির্বাসনে নিয়ে যায়।",NaN,NaN


In [28]:
# Loop through each query, invoke response and store in responses list
for query in queries:
    response = chain.invoke(query)  # Replace with your actual chain.invoke(Query) function
    responses.append(response)

In [29]:
# Completing DataFrame
df['answer'] = responses
df['contexts'] = retrieved_contexts

C:\Users\hp\AppData\Local\Temp\ipykernel_2120\3643552263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = responses
C:\Users\hp\AppData\Local\Temp\ipykernel_2120\3643552263.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contexts'] = retrieved_contexts


In [30]:
pd.set_option('display.max_colwidth', None)
df

,question,ground_truth,contexts,answer
0,মহাভারতের শুরুতে হস্তিনাপুরের শাসক কে ছিলেন?,রাজা শান্তনু।,"[হস্তিনাপুরের প্রাচীন রাজ্যে, একটি মহান মহাকাব্য উদ্ভাসিত হয়েছিল, যা বিশ্বের ভাগ্যকে রূপ দেবে এবং, অঙ্গের রাজার মুকুট পরিয়েছিলেন।]",উত্তর: মহাভারতের শুরুতে হস্তিনাপুরের শাসক অঙ্গের রাজা ছিলেন।
1,ভীষ্ম কী ব্রত নিয়েছিলেন এবং কেন?,ভীষ্ম তার পিতার সুখ নিশ্চিত করতে এবং তার পিতা শান্তনুকে সত্যবতীকে বিয়ে করার অনুমতি দেওয়ার জন্য ব্রহ্মচর্যের ব্রত নিয়েছিলেন।,"[যা পরে ভীষ্ম নামে পরিচিত হয়। ভীষ্ম তার পিতার সুখ নিশ্চিত করার জন্য ব্রহ্মচর্যের ব্রত নিয়েছিলেন,, রক্ষার ব্রত দ্বারা আবদ্ধ ভীষ্ম বীরত্বের সাথে যুদ্ধ করেছিলেন কিন্তু শিখণ্ডীর দ্বারা পরিচালিত]",ভীষ্ম ব্রহ্মচর্যের ব্রত নিয়েছিলেন তার পিতার সুখ নিশ্চিত করার জন্য।
2,রাজা শান্তনু ও সত্যবতীর পুত্র কারা ছিলেন?,চিত্রাঙ্গদা ও বিচিত্রবীর্য।,"[অঙ্গের রাজার মুকুট পরিয়েছিলেন।, রাজা শান্তনু শাসিত কুরু রাজবংশের সাথে গল্পটি শুরু হয়েছিল। শান্তনু নদী দেবী গঙ্গার সাথে বিবাহ]",সত্যবতীর পুত্র ছিলেন ভীষ্ম।
3,ধৃতরাষ্ট্র ও পাণ্ডুর অস্তিত্ব কীভাবে হয়েছিল?,"ধৃতরাষ্ট্র এবং পাণ্ডুর জন্ম নিয়োগের মাধ্যমে, একটি অনুশীলন যেখানে ব্যাস বিচিত্রবীর্যের মৃত্যুর কারণে রাণী অম্বিকা এবং অম্বালিকার সাথে সন্তানের জন্ম দিয়েছিলেন।","[ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়েছিলেন।, তাদের নির্বাসন এবং ছদ্মবেশী সময়ের পরে, পান্ডবরা তাদের রাজ্য ফিরে চেয়েছিল। কূটনৈতিক প্রচেষ্টা]",উত্তর: ধৃতরাষ্ট্র ও পাণ্ডুর অস্তিত্ব হয়েছিল ধর্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়ে।
4,মহাভারতে কুন্তীর ঐশ্বরিক আশীর্বাদের তাৎপর্য কী ছিল?,"কুন্তী দেবতাদের থেকে পুত্র সন্তান জন্ম দেওয়ার জন্য ঐশ্বরিক বর ব্যবহার করেছিলেন, এইভাবে পাণ্ডবদের জন্ম নিশ্চিত করেছিলেন: যুধিষ্ঠির, ভীম এবং অর্জুন।","[ও কৌরবদের মধ্যে মহাকাব্যিক অনুপাতের যুদ্ধ।, ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়েছিলেন।]",ঐশ্বরিক আশীর্বাদের তাৎপর্য ছিল মহাভারতে ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দেওয়া।
5,পাঁচ পান্ডব কারা ছিলেন এবং তাদের মা কে ছিলেন?,"পাঁচ পাণ্ডব ছিলেন যুধিষ্ঠির (ধর্মের পুত্র), ভীম (বায়ুর পুত্র), অর্জুন (ইন্দ্রের পুত্র), নকুল এবং সহদেব (অশ্বিনী যমজের পুত্র)। কুন্তী ছিলেন যুধিষ্ঠির, ভীম ও অর্জুনের মা, আর মাদ্রী ছিলেন নকুল ও সহদেবের মা।","[নকুল ও সহদেবের জন্ম দেন। এই পাঁচজনই ছিলেন পাণ্ডব।, তাদের নির্বাসন এবং ছদ্মবেশী সময়ের পরে, পান্ডবরা তাদের রাজ্য ফিরে চেয়েছিল। কূটনৈতিক প্রচেষ্টা]","পাঁচ পান্ডব ছিলেন নকুল, সহদেব, যুধিষ্ঠির, ভীম এবং অর্জুন। তাদের মা কুন্তী ছিলেন।"
6,দ্রোণ কীভাবে কৌরব ও পাণ্ডব রাজকুমারদের শিক্ষক হলেন?,উন্নত সামরিক দক্ষতা প্রদানের জন্য দ্রোণকে তাদের শিক্ষক হিসাবে নিযুক্ত করা হয়েছিল।,"[রাজকুমারদের বয়স হওয়ার সাথে সাথে দ্রোণ, উন্নত সামরিক শিল্পে পারদর্শী, তাদের শিক্ষক নিযুক্ত হন।, ধর্ম সম্পর্কে গভীর শিক্ষা দিয়েছিলেন।]",দ্রোণ কৌরব ও পাণ্ডব রাজকুমারদের শিক্ষক হিসেবে তাদের উন্নত সামরিক শিল্পে পারদর্শী হিসেবে নিযুক্ত হন এবং ধর্ম সম্পর্কে গভীর শিক্ষা দিয়েছিলেন।
7,কি কারণে অর্জুন ও কর্ণের মধ্যে শত্রুতা হয়েছিল?,"কর্ণ, প্রাথমিকভাবে একজন অজানা যোদ্ধা, দ্রৌপদীর স্বয়ম্বরে অর্জুনকে চ্যালেঞ্জ করেছিলেন। কর্ণের অসন্তোষ তার রাজকীয় রক্ত ​​সত্ত্বেও তার নিম্ন লালন-পালন থেকে উদ্ভূত হয়েছিল এবং কর্ণের প্রতি দুর্যোধনের সমর্থন প্রতিদ্বন্দ্বিতাকে আরও গভীর করে তোলে।","[শিক্ষক নিযুক্ত হন। অর্জুন, তৃতীয় পান্ডব, ​​তার অতুলনীয় দক্ষতার কারণে দ্রোণের প্রিয় হয়ে ওঠেন।, প্রিয় হয়ে ওঠেন। অর্জুনের পরাক্রমের প্রতি দুর্যোধনের ঈর্ষান্বিত হয়ে বিবাদের বীজ বপন করে।]",অর্জুনের পরাক্রমের প্রতি দুর্যোধনের ঈর্ষান্বিত হয়ে বিবাদের বীজ বপন করার কারণে অর্জুন ও কর্ণের মধ্যে শত্রুতা হয়েছিল।
8,পাণ্ডবদের জীবনে দ্রৌপদীর ভূমিকা কী ছিল?,দ্রৌপদী তার স্বয়ম্বর সম্পর্কে ভুল বোঝাবুঝির কারণে পাঁচ পাণ্ডবেরই সাধারণ স্ত্রী হয়েছিলেন। কৌরব দরবারে তার অপমান ছিল একটি গুরুত্বপূর্ণ ঘটনা যা মহান যুদ্ধের দিকে নিয়ে যায়।,"[ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়েছিলেন।, বিয়েতে দ্রৌপদীর হাত জিতেছিলেন। দ্রৌপদী, একটি ভুল বোঝাবুঝির মাধ্যমে, তাদের বন্ধনকে সম্মান জানিয়ে]",দ্রৌপদী পাণ্ডবদের জীবনে একটি গুরুত্বপূর্ণ ভূমিকা পেরিয়েছিল। বিয়েতে দ্রৌপদীর হাত জিতেছিলেন এবং তার মাধ্যমে তাদের বন্ধনকে সম্মান জানানো হয়।
9,যুধিষ্ঠির ও দুর্যোধনের মধ্যে পাশা খেলার সময় কী ঘটেছিল?,"যুধিষ্ঠির তার রাজ্য, ভাই এবং দ্রৌপদী সহ দুর্যোধনের কাছে সবকিছু হারিয়েছিলেন, যা পাণ্ডবদের নির্বাসনে নিয়ে যায়।","[মহৎ নগরীতে ব

-------------------------------------------------------RAG SYSTEM COMPLETES HERE-----------------------------------------------------------

## Evaluation

In [31]:
! pip install ragas nest_asyncio --quiet

In [32]:
import os
from ragas import evaluate
import pandas as pd
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness
import nest_asyncio

nest_asyncio.apply()

In [33]:
# Create a Dataset object from the data samples 
evaluation_dataset = Dataset.from_pandas(df)
evaluation_dataset

Dataset({
    features: ['question', 'ground_truth', 'contexts', 'answer'],
    num_rows: 20
})

In [35]:
# Evaluate the dataset using the specified metrics
score = evaluate(evaluation_dataset, metrics=[faithfulness, answer_correctness])

# Convert the score to pandas DataFrame for better visualization
results = score.to_pandas()

Evaluating:   2%|▎         | 1/40 [00:03<02:04,  3.20s/it]Task exception was never retrieved
future: <Task finished name='Task-25' coro=<as_completed.<locals>.sema_coro() done, defined at c:\Tools\Python\Lib\site-packages\ragas\executor.py:35> exception=TimeoutError()>
Traceback (most recent call last):
  File "c:\Tools\Python\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Tools\Python\Lib\site-packages\ragas\metrics\_faithfulness.py", line 277, in _ascore
    await _faithfulness_output_parser.aparse(
  File "c:\Tools\Python\Lib\site-packages\ragas\llms\output_parser.py", line 67, in aparse
    output = await llm.generate(p_value)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Tools\Python\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Tools\Python\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapp

In [36]:
results.to_csv('results_bn.csv', index=False)
results

,question,ground_truth,contexts,answer,faithfulness,answer_correctness
0,মহাভারতের শুরুতে হস্তিনাপুরের শাসক কে ছিলেন?,রাজা শান্তনু।,"[হস্তিনাপুরের প্রাচীন রাজ্যে, একটি মহান মহাকাব্য উদ্ভাসিত হয়েছিল, যা বিশ্বের ভাগ্যকে রূপ দেবে এবং, অঙ্গের রাজার মুকুট পরিয়েছিলেন।]",উত্তর: মহাভারতের শুরুতে হস্তিনাপুরের শাসক অঙ্গের রাজা ছিলেন।,0.000000,0.818059
1,ভীষ্ম কী ব্রত নিয়েছিলেন এবং কেন?,ভীষ্ম তার পিতার সুখ নিশ্চিত করতে এবং তার পিতা শান্তনুকে সত্যবতীকে বিয়ে করার অনুমতি দেওয়ার জন্য ব্রহ্মচর্যের ব্রত নিয়েছিলেন।,"[যা পরে ভীষ্ম নামে পরিচিত হয়। ভীষ্ম তার পিতার সুখ নিশ্চিত করার জন্য ব্রহ্মচর্যের ব্রত নিয়েছিলেন,, রক্ষার ব্রত দ্বারা আবদ্ধ ভীষ্ম বীরত্বের সাথে যুদ্ধ করেছিলেন কিন্তু শিখণ্ডীর দ্বারা পরিচালিত]",ভীষ্ম ব্রহ্মচর্যের ব্রত নিয়েছিলেন তার পিতার সুখ নিশ্চিত করার জন্য।,1.000000,0.618101
2,রাজা শান্তনু ও সত্যবতীর পুত্র কারা ছিলেন?,চিত্রাঙ্গদা ও বিচিত্রবীর্য।,"[অঙ্গের রাজার মুকুট পরিয়েছিলেন।, রাজা শান্তনু শাসিত কুরু রাজবংশের সাথে গল্পটি শুরু হয়েছিল। শান্তনু নদী দেবী গঙ্গার সাথে বিবাহ]",সত্যবতীর পুত্র ছিলেন ভীষ্ম।,0.000000,0.216027
3,ধৃতরাষ্ট্র ও পাণ্ডুর অস্তিত্ব কীভাবে হয়েছিল?,"ধৃতরাষ্ট্র এবং পাণ্ডুর জন্ম নিয়োগের মাধ্যমে, একটি অনুশীলন যেখানে ব্যাস বিচিত্রবীর্যের মৃত্যুর কারণে রাণী অম্বিকা এবং অম্বালিকার সাথে সন্তানের জন্ম দিয়েছিলেন।","[ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়েছিলেন।, তাদের নির্বাসন এবং ছদ্মবেশী সময়ের পরে, পান্ডবরা তাদের রাজ্য ফিরে চেয়েছিল। কূটনৈতিক প্রচেষ্টা]",উত্তর: ধৃতরাষ্ট্র ও পাণ্ডুর অস্তিত্ব হয়েছিল ধর্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়ে।,NaN,0.219512
4,মহাভারতে কুন্তীর ঐশ্বরিক আশীর্বাদের তাৎপর্য কী ছিল?,"কুন্তী দেবতাদের থেকে পুত্র সন্তান জন্ম দেওয়ার জন্য ঐশ্বরিক বর ব্যবহার করেছিলেন, এইভাবে পাণ্ডবদের জন্ম নিশ্চিত করেছিলেন: যুধিষ্ঠির, ভীম এবং অর্জুন।","[ও কৌরবদের মধ্যে মহাকাব্যিক অনুপাতের যুদ্ধ।, ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়েছিলেন।]",ঐশ্বরিক আশীর্বাদের তাৎপর্য ছিল মহাভারতে ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দেওয়া।,1.000000,0.224582
5,পাঁচ পান্ডব কারা ছিলেন এবং তাদের মা কে ছিলেন?,"পাঁচ পাণ্ডব ছিলেন যুধিষ্ঠির (ধর্মের পুত্র), ভীম (বায়ুর পুত্র), অর্জুন (ইন্দ্রের পুত্র), নকুল এবং সহদেব (অশ্বিনী যমজের পুত্র)। কুন্তী ছিলেন যুধিষ্ঠির, ভীম ও অর্জুনের মা, আর মাদ্রী ছিলেন নকুল ও সহদেবের মা।","[নকুল ও সহদেবের জন্ম দেন। এই পাঁচজনই ছিলেন পাণ্ডব।, তাদের নির্বাসন এবং ছদ্মবেশী সময়ের পরে, পান্ডবরা তাদের রাজ্য ফিরে চেয়েছিল। কূটনৈতিক প্রচেষ্টা]","পাঁচ পান্ডব ছিলেন নকুল, সহদেব, যুধিষ্ঠির, ভীম এবং অর্জুন। তাদের মা কুন্তী ছিলেন।",0.500000,0.509499
6,দ্রোণ কীভাবে কৌরব ও পাণ্ডব রাজকুমারদের শিক্ষক হলেন?,উন্নত সামরিক দক্ষতা প্রদানের জন্য দ্রোণকে তাদের শিক্ষক হিসাবে নিযুক্ত করা হয়েছিল।,"[রাজকুমারদের বয়স হওয়ার সাথে সাথে দ্রোণ, উন্নত সামরিক শিল্পে পারদর্শী, তাদের শিক্ষক নিযুক্ত হন।, ধর্ম সম্পর্কে গভীর শিক্ষা দিয়েছিলেন।]",দ্রোণ কৌরব ও পাণ্ডব রাজকুমারদের শিক্ষক হিসেবে তাদের উন্নত সামরিক শিল্পে পারদর্শী হিসেবে নিযুক্ত হন এবং ধর্ম সম্পর্কে গভীর শিক্ষা দিয়েছিলেন।,0.333333,0.658761
7,কি কারণে অর্জুন ও কর্ণের মধ্যে শত্রুতা হয়েছিল?,"কর্ণ, প্রাথমিকভাবে একজন অজানা যোদ্ধা, দ্রৌপদীর স্বয়ম্বরে অর্জুনকে চ্যালেঞ্জ করেছিলেন। কর্ণের অসন্তোষ তার রাজকীয় রক্ত ​​সত্ত্বেও তার নিম্ন লালন-পালন থেকে উদ্ভূত হয়েছিল এবং কর্ণের প্রতি দুর্যোধনের সমর্থন প্রতিদ্বন্দ্বিতাকে আরও গভীর করে তোলে।","[শিক্ষক নিযুক্ত হন। অর্জুন, তৃতীয় পান্ডব, ​​তার অতুলনীয় দক্ষতার কারণে দ্রোণের প্রিয় হয়ে ওঠেন।, প্রিয় হয়ে ওঠেন। অর্জুনের পরাক্রমের প্রতি দুর্যোধনের ঈর্ষান্বিত হয়ে বিবাদের বীজ বপন করে।]",অর্জুনের পরাক্রমের প্রতি দুর্যোধনের ঈর্ষান্বিত হয়ে বিবাদের বীজ বপন করার কারণে অর্জুন ও কর্ণের মধ্যে শত্রুতা হয়েছিল।,0.000000,0.438693
8,পাণ্ডবদের জীবনে দ্রৌপদীর ভূমিকা কী ছিল?,দ্রৌপদী তার স্বয়ম্বর সম্পর্কে ভুল বোঝাবুঝির কারণে পাঁচ পাণ্ডবেরই সাধারণ স্ত্রী হয়েছিলেন। কৌরব দরবারে তার অপমান ছিল একটি গুরুত্বপূর্ণ ঘটনা যা মহান যুদ্ধের দিকে নিয়ে যায়।,"[ধার্মিকতা এবং জীবনের অস্থিরতার উপর জোর দিয়েছিলেন।, বিয়েতে দ্রৌপদীর হাত জিতেছিলেন। দ্রৌপদী, একটি ভুল বোঝাবুঝির মাধ্যমে, তাদের বন্ধনকে সম্মান জানিয়ে]",দ্রৌপদী পাণ্ডবদের জীবনে একটি গুরুত্বপূর্ণ ভূমিকা পেরিয়েছিল। বিয়েতে দ্রৌপদীর হাত জিতেছিলেন এবং তার মাধ্যমে তাদের বন্ধনকে সম্মান জানানো হয়।,0.333333,0.2286